<a href="https://colab.research.google.com/github/amitrke/verizon-bill-parser/blob/main/vzw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies

In [ ]:
pip install --ignore-installed git+https://github.com/amitrke/verizon-bill-parser.git

Mount Google Drive

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')





Get Bill details

In [ ]:
from verizon_bill_parser import parser
import pandas as pd
import os

folder_path = '/content/drive/MyDrive/Colas/Vow/'

try:
  parsedFileList = parser.parse_directory(folder_path)
  for parsedFile in parsedFileList:
    print(f"\n*** FILE *** ${parsedFile['fileName']} ****\n")
    df = pd.DataFrame(parsedFile['amounts'])
    print(df)

except Exception as e:
  print(e)


Get Bill details for a specific phone number

In [ ]:
from verizon_bill_parser import parser
import pandas as pd
import logging

# Suppress noisy logs
parser.set_logger_level('ERROR')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

folder_path = '/content/drive/MyDrive/Colab/vzw-2025/'
phoneNum = '123-456-7890'  # Replace with the desired phone number

rows = []

parsedFileList = parser.parse_directory(folder_path)

name=""
for parsedFile in parsedFileList:
    bill_date = parsedFile['billDate'] 
    for amount in parsedFile['amounts']:
        if amount.get("phoneNum") == phoneNum:
          if name=="":
            name=amount.get("name")
          rows.append({
              "billDate": bill_date,
              "amount": float(amount['amount'].replace('$', ''))
          })

# Build DataFrame
df = pd.DataFrame(rows)

# Convert billDate to datetime (MM/DD/YYYY)
df["billDate"] = pd.to_datetime(df["billDate"], format="%m/%d/%Y")

# Month-wise aggregation
monthly_summary = (
    df.groupby("billDate", as_index=False)
      .sum()
      .rename(columns={"amount": "total_amount"})
      .sort_values("billDate")
)

# Optional: format date nicely for display
monthly_summary["billDate"] = monthly_summary["billDate"].dt.strftime("%m/%d/%Y")

# Grand total
grand_total = monthly_summary["total_amount"].sum()

print(f"\nPhone Number: {phoneNum}; Name: {name}")
print(monthly_summary)
print(f"\nGrand Total: ${grand_total:.2f}")